In [2]:
import numpy as np
import scipy.linalg as LA
import pandas as pd
import copy
import os
import time
import importlib
import sys
import matplotlib.pyplot as plt
import matplotlib as mpl
from matplotlib.path import Path
import matplotlib.patches as patches
from matplotlib.colors import ListedColormap
from matplotlib import ticker

import convert_hr_file as HRtoDF
#importlib.reload()

In [3]:
from joblib import Parallel, delayed
import functools

def parallel( iteration, Njob, Nver, func  ):
    return Parallel(n_jobs=Njob,verbose=Nver)( [delayed(func)(i) for i in iteration ] )

In [17]:
prefix = "xxx"
hr_file = "./xxx_hr.dat"
n_skip = 0 # # of the header part to skip
fermi = 0
# input the same structure with the .win file: bravais vectors, unitcell, orbitals

bravais = np.array([ [ 0.866025403784439,  -0.500000000000000,   0.000000000000000 ],
                    [0.000000000000000,   1.000000000000000,   0.000000000000000 ],
                    [ 0.000000000000000,   0.000000000000000,   1 ] ])
unitcell    = np.array([ [ 0.333333333333333 ,  0.666666666666667,   0.250000000000000 ],
                        [ 0.666666666666667,   0.333333333333333,   0.750000000000000 ] ])
unitcell = [ np.dot(bravais.T,r) for r in unitcell ]
orbital_list = ["dzx","dyz","dxy","dz2","dx2y2"]

size = 20 # row(=column) size of the Hamiltonian H(k) 

# symmetric points in the wave number space
k_labels = ["Γ","X"]
G = [ 0, 0, 0 ]
X = [ 0.5, 0, 0 ]
k_symm = np.array([ G,X ])

#be careful of the order of orbital
df_hop = HRtoDF.hr_file_to_df_hop( hr_file, n_skip, bravais, unitcell, orbital_list )
output_band = prefix + "_wf_band.csv"

In [6]:
def reciprocal( bravais_T ):
    reciprocal = np.zeros((3,3))
    deno = LA.det( bravais_T.T )
    for i in [0,1,2]:
        reciprocal[i] = np.cross( bravais_T[(i+1)%3], bravais_T[(i+2)%3] )/deno
    return reciprocal

def k_xyz_arr_to_k_crys_arr( k_xyz_arr, bravais ):
    recip = reciprocal( bravais )
    return np.array([ np.dot( LA.inv(recip).T, k ).tolist() for k in k_xyz_arr ])

def add_k_on_way( k_arr0, N ):
    k_list = []
    for i in range( len(k_arr0) - 1 ):
        dk = ( k_arr0[i+1]-k_arr0[i] )/N
        for n in range(N):
            k_list += [ (k_arr0[i] + n*dk).tolist() ]
    k_list += [ k_arr0[-1].tolist() ]
    return np.array(k_list)


def calc_k_trace( k_arr, bravais ):
    recip = reciprocal( bravais )
    k_sum = 0
    k_trace_list = [0]
    for i in range( 1, len(k_arr) ):
        k_sum += LA.norm( np.dot(recip.T,k_arr[i]-k_arr[i-1]), 2 )
        k_trace_list += [k_sum]
    if( len(k_trace_list) > 1 ): k_trace_list = [ K/k_trace_list[-1] for K in k_trace_list ]
    return k_trace_list

def make_hamiltonian_list( k_arr, df_hop, size, fermi ):
    hami_list = [ np.zeros((size,size),dtype=np.complex128) for i in range( len(k_arr) ) ]
    for n_row,sr in df_hop[ abs(df_hop["hop"])>1e-4 ].iterrows():
        an = np.array([ sr['a1'], sr['a2'], sr['a3'] ])
        t = sr['hop']
        row = sr['n_band1']
        col = sr['n_band2']
        for i, k in enumerate( k_arr ):
            phase = 2*np.pi*np.dot(an,k)
            hami_list[i][row,col] += t*( np.cos(phase) + 1j*np.sin(phase) )        
    return [ h - fermi*np.identity(size) for h in hami_list ]

def calc_band( k_arr, df_hop, size, fermi, bravais ):
    start = time.time()
    hami_list = make_hamiltonian_list( k_arr, df_hop, size, fermi )
    #band_list = [ LA.eigvalsh( h ).tolist() for h in hami_list ]
    band_list = Parallel(n_jobs=-2,verbose=4)( [delayed(LA.eigvalsh)(h) for h in hami_list ] )
    band_list = [ Ek.tolist() for Ek in band_list ]
    end = time.time()
    print( "{0} sec.".format( end - start ) )
    return pd.DataFrame( band_list, index = calc_k_trace( k_arr, bravais ) )

In [7]:
# add intermidiate points between symmetric points in the k space
k_arr = add_k_on_way( k_symm, 20 )

In [ ]:
df_band = calc_band( k_arr, df_hop, size, fermi, bravais )

In [ ]:
df_band

In [138]:
df_band.to_csv(output_band)

In [ ]:
y_RNG = [-3,3]
symm_point_list = ["Γ","X"]
symm_position_list = [ 0, 0.5774 ]
symm_position_list = [ k/symm_position_list[-1] for k in symm_position_list ]

fig = plt.figure(figsize=(8,5))
ax = fig.add_subplot(111)

ax = df_band.plot(legend=False,color="red",lw=1,ax=ax)

ax.set_xlabel("")
ax.set_ylabel("Energy [eV]",fontsize=18)
ax.set_xticks(symm_position_list)
#ax.set_xticklabels(symm_point_list)
ax.tick_params(labelsize=18)
ax.axhline(y=0,color='black',linestyle=':')
for k in symm_position_list:
    ax.axvline(x=k,color='black',linestyle=':')
ax.set_xlim([symm_position_list[0],symm_position_list[-1]])
ax.tick_params(direction='in', length=6, width=2,pad=10)
plt.tight_layout()
plt.ylim(y_RNG)
fig.savefig( prefix+".band_wf.pdf",bbox_inches="tight", pad_inches=0.0, dpi=300)